In [ ]:
import datetime as dt
import numpy as np
import cvxpy
from cvxopt import solvers
from alphamind.portfolio.linearbuilder import linear_build

In [ ]:
solvers.options['glpk'] = {'msg_lev': 'GLP_MSG_OFF'}

In [ ]:
def time_function(py_callable, n):
    start = dt.datetime.now()
    result = py_callable(n)
    elapsed = (dt.datetime.now() - start).total_seconds()
    return elapsed, result

def cvxpy_lp(n):
    w = cvxpy.Variable(n)

    bndl = np.zeros(n)
    bndu = 0.01 * np.ones(n)
    risk_constraints = np.ones((n,1))

    curr_risk_exposure = risk_constraints.T @ w

    constraints = [w >= bndl,
                   w <= bndu,
                   curr_risk_exposure >= 1.,
                   curr_risk_exposure <= 1.]

    er = np.random.randn(n)


    objective = cvxpy.Minimize(-w.T * er)
    prob = cvxpy.Problem(objective, constraints)
    prob.solve(solver='GLPK')
    return w, prob

In [ ]:
print("{0:<8}{1:>12}{2:>12}{3:>12}{4:>12}{5:>12}".format('Scale(n)', 'time(ms)', 'feval', 'min(x)', 'max(x)', 'sum(x)'))

for n in range(200, 3200, 200):
    elapsed, result = time_function(cvxpy_lp, n)
    s = np.array(result[0].value).flatten()
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12f}{4:>12f}{5:>12f}".format(n, elapsed*1000, result[1].value, s.min(), s.max(), s.sum()))

In [ ]:
def clp_lp(n):
    er = np.random.randn(n)

    bndl = np.zeros(n)
    bndu = 0.01 * np.ones(n)
    risk_constraints = np.ones((n,1))
    risk_target = np.ones(1), np.ones(1)
    
    result = linear_build(er, bndl, bndu, risk_constraints, risk_target)
    return result

In [ ]:
print("{0:<8}{1:>12}{2:>12}{3:>12}{4:>12}{5:>12}".format('Scale(n)', 'time(ms)', 'feval', 'min(x)', 'max(x)', 'sum(x)'))

for n in range(200, 3200, 200):
    elapsed, result = time_function(clp_lp, n)
    s = result[2]
    print("{0:<8}{1:>12.2f}{2:>12.2f}{3:>12f}{4:>12f}{5:>12f}".format(n, elapsed*1000, result[1], s.min(), s.max(), s.sum()))